<a href="https://colab.research.google.com/github/italolima04/monitoria-probabilidade-estatistica/blob/master/Pesquisa-PID/Implementa%C3%A7%C3%A3o-Algoritmos/Apresenta%C3%A7%C3%A3o_PID_Implementa%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aprendizado Não-Supervisionado

"*O aprendizado não supervisionado é um ramo do Machine Learning que aprende com dados de teste que não foram rotulados, classificados ou categorizados previamente. Em vez de responder à programação de um operador, o aprendizado não supervisionado identifica semelhanças nos dados e reage com base na presença ou ausência de tais semelhanças em cada novo dado*

**Essa abordagem de Aprendizado de Máquina é útil quando não possuímos rótulos (Labels) para os nossos dados. Isto é importante pelo fato de que em muitos contexos possuir esses dados pode ser difícil e/ou custoso.**


Nesse caso de estudo, pelo fato da base de dados ser atual, real e ter sido coletada em um contexto de uma rede social, não se faz possível possuir rótulos para a classificação de textos. Dessa forma, objetiva-se agrupar os dados de forma não supervisionada.

# Clustering

**Clustering é uma técnica de Aprendizado de Máquina que envolve o agrupamento de pontos de dados.**

Utiliza-se algoritmos de clustering para agrupar pontos de dados em grupos específicos, cujos, na teoria devem possuir propriedades/características semelhantes.

**K-Means**

K-Means é um método de Clustering que tem como objetivo o particionamento de n observações dentre k grupos, onde cada observação pertence ao grupo mais próximo da média.


A execução do K-Means segue um conjunto de passos, descritos abaixo:

1. Selecionar um número de classes/grupos para utilizar e inicializarmos aleatoriamente seus respectivos pontos centrais (Centróides). Esses grupos podem ser definidos de acordo com a regra do negócio ou a perspectiva do problema.
2. Cada de ponto de dados é classificado, baseando-se na distância entre esse ponto e o centro do grupo.
3. Com base nos pontos classificados, recalcula-se o centróide, a partir da média das distâncias de todos os vetores do grupo.  

Esses dois últimos passos são repetidos, até que o limite de iterações pré-determinado seja atingido, ou quando os centróides não sofrerem alterações significantes de uma iteração para outra.



**Vantagens:**

- É rápido, visto que o que é feito é somente calcular as distâncias entre os pontos e os centróides do grupo. Possui então complexidade linear O(n). 

- É simples de aplicar e se mostra eficiente.

**Desvantagens:**

- Dificuldade para determinar a quantidade de clusters (Dependendo do Objetivo).

- Pode ser inconsistente, a depende do conjunto de dados.

# Importando Bibliotecas, Módulos e Dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
import re
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#Criando Data Frame a partir dos dados.
data = pd.read_csv('dados-pesquisa.csv')

# Visualizando e Explorando os dados

In [ ]:
#Visualizando as 5 primeiras linhas.
data.head()

,Unnamed: 0,created_at,id,text,user,lang
0,0,Mon May 18 17:53:01 +0000 2020,NaN,"Tudo mudou na nossa forma de trabalhar, mas na...",NaN,pt
1,1,Mon May 18 17:52:17 +0000 2020,NaN,McDonald’s já reabriu lojas ao público https:/...,NaN,pt
2,2,Mon May 18 17:50:40 +0000 2020,NaN,"Sindicato, está atento em tudo o que envolve s...",NaN,pt
3,3,Mon May 18 17:49:17 +0000 2020,NaN,Nossos problemas da saúde definitivamente acab...,NaN,pt
4,4,Mon May 18 17:49:06 +0000 2020,NaN,Fábricas de todo o mundo se viram obrigadas a ...,NaN,pt


In [ ]:
#Visualizando as 5 últimas linhas.
data.tail()

,Unnamed: 0,created_at,id,text,user,lang
109959,109959,Mon Jul 20 15:04:14 +0000 2020,1.285229e+18,porto alegre perigando a entrar em lockdown e ...,"{'id': 1245313171, 'id_str': '1245313171', 'na...",pt
109960,109960,Mon Jul 20 15:03:35 +0000 2020,1.285229e+18,"Nunca que a Mídia vai falar isso!\nAMB, CFM e ...","{'id': 40697641, 'id_str': '40697641', 'name':...",pt
109961,109961,Mon Jul 20 15:03:33 +0000 2020,1.285229e+18,@joaopiresrj Eu não sou cientista mas esse LOC...,"{'id': 942176977, 'id_str': '942176977', 'name...",pt
109962,109962,Mon Jul 20 15:01:54 +0000 2020,1.285228e+18,Q&amp;A - Recessão ou Lockdown. O que é pior? ...,"{'id': 1136982345137958912, 'id_str': '1136982...",pt
109963,109963,Mon Jul 20 15:01:38 +0000 2020,1.285228e+18,O prefeito de Los Angeles está preste a decret...,"{'id': 494366716, 'id_str': '494366716', 'name...",pt


In [ ]:
#Verificando a estrutura inicial dos dados.
data.shape

(109964, 6)

**Podemos observar aproximadamente 110 mil linhas e 6 colunas.**

# Tratando os dados

In [ ]:
#Removendo dados duplicados na coluna de Texto e substituindo dentro do próprio Data Frame.
data.drop_duplicates(['text'], inplace=True)

In [ ]:
#Verificando novamente a estrutura dos dados para observar a quantidade de dados únicos.
data.shape

(103364, 6)

6600 linhas foram removidas.

**Iremos trabalhar com os Dados de Texto, por isso, selecionaremos apenas a coluna associada ao conteúdo dos tweets.**

In [ ]:
#Utilizando notação comum em estudos de Aprendizado de Máquina.
X = data['text']

In [ ]:
#Visualizando as 15 primeiras linhas. 
X[:15]

0     Tudo mudou na nossa forma de trabalhar, mas na...
1     McDonald’s já reabriu lojas ao público https:/...
2     Sindicato, está atento em tudo o que envolve s...
3     Nossos problemas da saúde definitivamente acab...
4     Fábricas de todo o mundo se viram obrigadas a ...
5     @g1 @RedeGlobo @jornalhoje o governo de @jairb...
6     Lei N° 6666, nos #EUA, que pretende traçar e i...
7     https://t.co/mcftUwJr5F o governo precisa ser ...
8     Blockchain: a tecnologia que popularizou o #bi...
9     VOCÊS CONHECEM ALGUM PAÍS ALÉM DO BRASIL QUE T...
10    Itajuípe recebe o Centro Municipal de Isolamen...
11    📣 NOTÍCIA / NEWS / NOUVELLES \n\n🇵🇹 Manual de ...
12    Enquanto o governo Bolsonaro permanecer Irresp...
13    Faça a diferença! Seja um doador sem fronteira...
14    Manaus, maio de 2020. Reportagem sobre o colap...
Name: text, dtype: object

**Devido ao fato de os dados serem proveninentes de uma rede social, se faz necessária uma etapa de pré-processamento nos mesmos.**

In [ ]:
def Preprocessing_data(instance):
      instance = re.sub(r"http\S+", "", instance).lower().replace('.', '').replace(';','').replace('-','').replace(':', '').replace(')', '')
      stopwords = set(nltk.corpus.stopwords.words('portuguese'))
      words = [i for i in instance.split() if not i in stopwords]
      return (" ".join(words))

**Função que remove links, urls, sinais de pontuação, padroniza os caracteres como minúsculos e remove as stopwords através de um dos módulos da biblioteca NLTK.**

In [ ]:
#Aplicando a função na nossa base de dados.
X = [Preprocessing_data(i) for i in X]

In [ ]:
#Visualizando novamente as 15 primeiras linhas, agora após o pré-processamento.
X[:15]

['tudo mudou forma trabalhar, nada mudou objetivo informar #jornalismo #imprensa…',
 'mcdonald’s reabriu lojas público #comida #covid19 #mcdonalds',
 'sindicato, atento tudo envolve saúde condições trabalho nessa pandemia #covid @spbancarios',
 'problemas saúde definitivamente acabaram #saude #forabolsonaro #covid',
 'fábricas todo mundo viram obrigadas dispensarem funcionários decorrência #covid19 antes m…',
 '@g1 @redeglobo @jornalhoje governo @jairbolsonaro vai socorrer ninguém vai enrolar! pois minto quer r…',
 'lei n° 6666, #eua, pretende traçar investigar percurso pessoas tivestes contatos, usand…',
 'governo precisa ser responsabilizado #covid #covid19',
 'blockchain tecnologia popularizou #bitcoin sendo testada combate #covid19 sistema capaz de…',
 'conhecem algum país além brasil torcida organizada coronavirus???? #covid #covid19',
 'itajuípe recebe centro municipal isolamento covid19 #covid19 #isolamentosocial #covid @rctitajuipe',
 '📣 notícia / news / nouvelles 🇵🇹 manual boa

**É atribuído um número para cada palavra encontrada na base de dados.**

# Criando o Modelo Word2VEC.

In [ ]:
#Importando biblioteca e módulo do Gensim, para a implementação do Word2VEC.
import multiprocessing

from gensim.models import Word2Vec

In [ ]:
#Função que conta o número de cores da máquina.
cores = multiprocessing.cpu_count()

In [ ]:
#Instanciando modelo Word2VEC a partir dos mesmos parâmetros estabelecidos pelo artigo.
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     size=300,
                     sample=0.8, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

In [ ]:
#Construindo vocabulário a partir das sentenças.
w2v_model.build_vocab(sentences, progress_per=10000)

INFO - 15:13:08: collecting all words and their counts
INFO - 15:13:08: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 15:13:08: PROGRESS: at sentence #10000, processed 98386 words, keeping 26703 word types
INFO - 15:13:08: PROGRESS: at sentence #20000, processed 194307 words, keeping 41176 word types
INFO - 15:13:09: PROGRESS: at sentence #30000, processed 289323 words, keeping 53198 word types
INFO - 15:13:09: PROGRESS: at sentence #40000, processed 384343 words, keeping 63822 word types
INFO - 15:13:09: PROGRESS: at sentence #50000, processed 480594 words, keeping 73460 word types
INFO - 15:13:10: PROGRESS: at sentence #60000, processed 577296 words, keeping 82906 word types
INFO - 15:13:10: PROGRESS: at sentence #70000, processed 672253 words, keeping 91879 word types
INFO - 15:13:10: PROGRESS: at sentence #80000, processed 765282 words, keeping 100227 word types
INFO - 15:13:11: PROGRESS: at sentence #90000, processed 858298 words, keeping 107673 word typ

# Treinando o Modelo Word2VEC

In [ ]:
#Treiando modelo Word2VEC.
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

INFO - 15:13:19: training model with 1 workers on 30785 vocabulary and 300 features, using sg=0 hs=0 sample=0.8 negative=20 window=4
INFO - 15:13:20: EPOCH 1 - PROGRESS: at 4.74% examples, 40726 words/s, in_qsize 2, out_qsize 0
INFO - 15:13:21: EPOCH 1 - PROGRESS: at 12.85% examples, 53578 words/s, in_qsize 1, out_qsize 0
INFO - 15:13:22: EPOCH 1 - PROGRESS: at 20.95% examples, 59079 words/s, in_qsize 1, out_qsize 0
INFO - 15:13:23: EPOCH 1 - PROGRESS: at 29.08% examples, 62017 words/s, in_qsize 1, out_qsize 0
INFO - 15:13:24: EPOCH 1 - PROGRESS: at 37.26% examples, 62779 words/s, in_qsize 1, out_qsize 0
INFO - 15:13:25: EPOCH 1 - PROGRESS: at 45.23% examples, 64092 words/s, in_qsize 1, out_qsize 0
INFO - 15:13:26: EPOCH 1 - PROGRESS: at 54.26% examples, 65402 words/s, in_qsize 1, out_qsize 0
INFO - 15:13:27: EPOCH 1 - PROGRESS: at 63.40% examples, 66297 words/s, in_qsize 1, out_qsize 0
INFO - 15:13:29: EPOCH 1 - PROGRESS: at 71.62% examples, 66785 words/s, in_qsize 1, out_qsize 0
INFO

(26567790, 29622330)

In [ ]:
#Tornando o modelo mais eficiente em questões de uso de memória.
w2v_model.init_sims(replace=True)

INFO - 15:19:24: precomputing L2-norms of word weight vectors


**A partir do Word2VEC podemos verificar a similaridade de outras palavras para determinados termos.**

In [ ]:
w2v_model.wv.most_similar(["vacina"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('vacina_contra', 0.5999780893325806),
 ('vacina_chinesa', 0.5734825134277344),
 ('tomar_vacina', 0.5448654890060425),
 ('"vacina', 0.5391110181808472),
 ('vacinas', 0.5349497199058533),
 ('doses_vacina', 0.5105684995651245),
 ('cura_pro', 0.4802631735801697),
 ('#vacina', 0.46188271045684814),
 ('testes_vacina', 0.4576757252216339),
 ('vacina_oxford', 0.4500918984413147)]

**Podemos determinar filtros para uma verificação de similaridade.**

In [ ]:
w2v_model.wv.most_similar(["recuperados"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('curados', 0.6696262359619141),
 ('pacientes_recuperados', 0.6550328731536865),
 ('pessoas_recuperadas', 0.6002197265625),
 ('pessoas_curadas', 0.5826815366744995),
 ('casos_ativos', 0.5721123218536377),
 ('vítimas_fatais', 0.5445606112480164),
 ('recuperados…', 0.5399713516235352),
 ('confirmações', 0.5295443534851074),
 ('mortes_confirmadas', 0.520463228225708),
 ('recuperados,', 0.5047532320022583)]

In [ ]:
w2v_model.wv.most_similar(["mortes"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('mortes_causadas', 0.750665545463562),
 ('novas_mortes', 0.704981803894043),
 ('mortes_confirmadas', 0.6422505974769592),
 ('mortes,', 0.6406204700469971),
 ('vidas_perdidas', 0.6357656717300415),
 ('mortos', 0.6347059011459351),
 ('mortes_diárias', 0.6335180401802063),
 ('óbitos', 0.6294437646865845),
 ('1300_mortes', 0.6261337995529175),
 ('mil_mortes', 0.6245402097702026)]

**Podemos compararar a taxa de similaridade para termos específicos.**

In [ ]:
w2v_model.wv.similarity("coronavirus", 'mortes')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


-0.38125765

In [ ]:
w2v_model.wv.similarity("coronavirus", 'casos')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


-0.40818962

In [ ]:
w2v_model.wv.similarity("pandemia", 'coronavirus')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


-0.10278622

# Aplicando K-Means

In [ ]:
#Importando algoritmo KMeans da biblioteca Sklearn.
from sklearn.cluster import KMeans

In [ ]:
#Carregando modelo Word2VEC.
word_vectors = w2v_model.wv
#Criando instância do algoritmo K-Means e passando como entrada os vetores originados pelo Word2VEC.
model = KMeans(n_clusters=2, init='k-means++', max_iter=1000, n_init=50).fit(X=word_vectors.vectors)

In [ ]:
word_vectors.similar_by_vector(model.cluster_centers_[0], topn=10, restrict_vocab=None)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('@guicidade', 0.9998173713684082),
 ('padaria,', 0.9998025894165039),
 ('@maumeirelles', 0.9997973442077637),
 ('antibiotico', 0.9997907876968384),
 ('critico', 0.9997900724411011),
 ('otaku', 0.9997895956039429),
 ('@prudmin', 0.9997768998146057),
 ('banaliza', 0.9997758865356445),
 ('influenza,', 0.9997735023498535),
 ('fatos,', 0.9997729063034058)]

In [ ]:
word_vectors.similar_by_vector(model.cluster_centers_[1], topn=10, restrict_vocab=None)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('6,5', 0.9983620643615723),
 ('#cbnlondrina', 0.9981963634490967),
 ('280', 0.9977450370788574),
 ('atu…', 0.9976952075958252),
 ('ajuste', 0.9976508617401123),
 ('mantémse', 0.9975696802139282),
 ('divulgou,', 0.9974309206008911),
 ('vilhena', 0.9974086284637451),
 ('veranópolis', 0.9972860813140869),
 ('concentra', 0.9972808361053467)]

In [ ]:
#word_vectors.similar_by_vector(model.cluster_centers_[2], topn=10, restrict_vocab=None)

In [ ]:
positive_cluster_center = model.cluster_centers_[0]
negative_cluster_center = model.cluster_centers_[1]
#neutral_cluster_center = model.cluster_centers_[2]

In [ ]:
#Verificando os vetores que foram entradas para o algoritmo, criados pelo modelo Word2VEC.
word_vectors.vectors

array([[-0.04830528, -0.0888536 ,  0.0121821 , ...,  0.02902986,
        -0.06335721, -0.04157186],
       [-0.01620184, -0.08371107, -0.02201089, ...,  0.00480282,
        -0.02898812, -0.00793971],
       [-0.03907719, -0.04570566,  0.07377318, ..., -0.06223137,
        -0.10727366,  0.07251507],
       ...,
       [-0.07516849, -0.02846682,  0.05661228, ..., -0.04819285,
        -0.09105309,  0.01191978],
       [-0.07529917, -0.02962228,  0.068182  , ..., -0.05000691,
        -0.10008409,  0.02093332],
       [-0.0758184 , -0.02824306,  0.06028139, ..., -0.04901572,
        -0.08991348,  0.01396022]], dtype=float32)

In [ ]:
model.cluster_centers_

array([[-7.21572861e-02, -2.74464209e-02,  6.33466169e-02,
         1.63704399e-02,  7.07888529e-02, -2.70095584e-03,
        -5.42753004e-02, -1.25905806e-02,  3.67578156e-02,
        -5.69887226e-03,  7.08400533e-02, -7.03297630e-02,
         9.57227312e-05, -6.25459924e-02,  5.14166765e-02,
         2.14814395e-02, -4.90327254e-02, -1.23748612e-02,
        -1.67479794e-02,  1.23052765e-02, -3.51850837e-02,
        -3.25769209e-03,  3.64111364e-02, -3.41699198e-02,
         1.56835943e-01,  8.15555975e-02, -1.29096329e-01,
         8.46559182e-02, -8.70009661e-02,  3.61581706e-02,
        -8.24830905e-02, -6.15216792e-02,  2.83797793e-02,
        -1.47375241e-01,  1.36916647e-02, -7.47230202e-02,
        -8.03454220e-02,  5.34621328e-02,  5.24857454e-02,
        -3.83956591e-03, -4.94446270e-02, -8.64642411e-02,
         2.88445177e-03, -7.58034214e-02,  5.08014001e-02,
         1.43853566e-02, -5.76101467e-02,  2.54720971e-02,
         2.34782360e-02,  3.54748368e-02,  1.42947969e-0

In [ ]:
len(model.labels_)

33032

In [ ]:
#Verificando os labels originados pelo modelo.
model.labels_

array([1, 1, 0, ..., 0, 0, 0], dtype=int32)

In [ ]:
#Criando DataFrame com as classificações para os tweets.
df_kmeans = pd.DataFrame(data=model.labels_, columns=['text'])

In [ ]:
#Visualizando o novo DataFrame criado.
df_kmeans

,text
0,1
1,1
2,0
3,0
4,1
...,...
33027,0
33028,0
33029,0
33030,0


In [ ]:
#Distruibuição das sentenças nos clusters pelo algoritmo K-Means.
df_kmeans['text'].value_counts()

0    25640
1     7392
Name: text, dtype: int64

In [ ]:
#Porcentagem que representa a distribuição das sentenças nos clusters pelo algoritmo K-Means.
df_kmeans['text'].value_counts(normalize=True)

0    0.776217
1    0.223783
Name: text, dtype: float64

In [ ]:
#Verificando os clusters únicos.
df_kmeans['text'].unique()

array([1, 0], dtype=int32)

In [ ]:
#Criando Vocabulário em formato de Dicionário para o sentimento das palavras.
words = pd.DataFrame(word_vectors.vocab.keys())
#Nomeando a coluna do DataFrame.
words.columns = ['words']
#Atribuindo as palavras a classe positiva ou negativa, de acordo com a clusterização.
words['vectors'] = words.words.apply(lambda x: word_vectors.wv[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


*Explicação para Função Lambda que atribui um sentimento em potencial para as palavras.*

**Para atribuir uma pontuação de sentimento para cada palavra foi realizada uma multiplicação pelo quão próximos eles estavam de seu cluster (para denotar o quão potencialmente positivos / negativos os termos são). Como a pontuação que o algoritmo K-means produz é a distância de ambos os clusters, para ponderá-los corretamente, foi feita a multiplicação pelo inverso da pontuação de proximidade (divisão da pontuação de sentimento pela pontuação de proximidade).**

In [ ]:
#Para o cluster 0 atribuiu-se o sentimento como positivo e para o outro cluster atribuiu-se o sentimento negativo.
words['cluster_value'] = [1 if i==0 else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

In [ ]:
words.head(5)

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
0,tudo,"[-0.055858582, -0.034655627, 0.07575433, 0.025...",0,1,1.317277,1.317277
1,mudou,"[-0.06306405, -0.02387166, 0.057486724, 0.0313...",0,1,3.083671,3.083671
2,forma,"[-0.056214195, -0.014575296, 0.05572673, 0.030...",0,1,2.737905,2.737905
3,"trabalhar,","[-0.062424235, -0.029514665, 0.07425506, 0.033...",0,1,1.929287,1.929287
4,nada,"[-0.05544726, -0.029607985, 0.08812149, 0.0292...",0,1,1.322934,1.322934


In [ ]:
words.tail(5)

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
33027,@bresciia,"[-0.07572785, -0.02133777, 0.06326089, 0.01852...",0,1,13.614501,13.614501
33028,@theluks21,"[-0.07558556, -0.028088983, 0.0642249, 0.01247...",0,1,15.027169,15.027169
33029,@kozixmana,"[-0.07529917, -0.02962228, 0.068182, 0.0175566...",0,1,13.216446,13.216446
33030,@victorkazoo,"[-0.07681176, -0.026911363, 0.0656198, 0.02408...",0,1,14.671299,14.671299
33031,#renunciaperes,"[-0.0758184, -0.028243065, 0.06028139, 0.01308...",0,1,11.633633,11.633633


In [ ]:
words.shape

(33032, 6)

In [ ]:
words[['words', 'sentiment_coeff']].to_csv('sentiment_dictionary.csv', index=False)

# Aplicando DBSCAN

In [ ]:
#Importando algoritmo da Biblioteca Sklearn.
from sklearn.cluster import DBSCAN

In [ ]:
word_vectors = w2v_model.wv

In [ ]:
#Criando instância do algoritmo e treinando o modelo com os dados.
clustering = DBSCAN(eps=0.5, min_samples=2).fit(X=word_vectors.vectors)

In [ ]:
#Verificando os labels originados.
clustering.labels_

array([-1, -1,  0, ...,  0,  0,  0])

In [ ]:
df_dbscan = pd.DataFrame(data=clustering.labels_, columns=['text'])

In [ ]:
df_dbscan

,text
0,-1
1,-1
2,0
3,0
4,-1
...,...
30780,0
30781,0
30782,0
30783,0


In [ ]:
df_dbscan['text'].value_counts()

 0     30653
-1        46
 5        12
 6         7
 12        6
 2         6
 14        6
 8         5
 4         5
 7         4
 11        4
 19        4
 1         4
 15        4
 13        3
 3         3
 10        3
 17        3
 16        3
 9         2
 18        2
Name: text, dtype: int64

In [ ]:
df_dbscan['text'].unique()

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19])

In [ ]:
#word_vectors.similar_by_vector(model.cluster_centers_[0], topn=10, restrict_vocab=None)

In [ ]:
#word_vectors.similar_by_vector(model.cluster_centers_[1], topn=10, restrict_vocab=None)

# Aplicando Mean Shift

In [ ]:
#Importando algoritmo da Biblioteca Sklearn.
#from sklearn.cluster import MeanShift

In [ ]:
#Criando instância do algoritmo.
#word_vectors = w2v_model.wv
#clustering = MeanShift(bandwidth=2).fit(X=word_vectors.vectors)

In [ ]:
#Verificando os labels originados.
#clustering.labels_

In [ ]:
#df_meanshift = pd.DataFrame(data=clustering.labels_, columns=['text'])

In [ ]:
#df_meanshift

In [ ]:
#df_meanshift['text'].value_counts()

In [ ]:
#df_meanshift['text'].unique()